# 위쪽 부분은 TEST, 아래쪽이 함수화한 부분입니다!

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings

plt.rc('font', family = 'Malgun Gothic')
filterwarnings('ignore')

# 전체 데이터 출력 및 확인 옵션 설정
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_info_columns

100

In [3]:
# place에 따라 자리가 다를 것으로 생각하고, 나눠서 작업하기
df = pd.read_csv("dataset/2023bigcontest_advanced_sac.csv")
df['place'].unique()

array(['콘서트홀', '리사이틀홀', 'IBK챔버홀'], dtype=object)

In [4]:
df_concert = df.query("`place`=='콘서트홀'")
df_chamber = df.query("`place`=='IBK챔버홀'")
df_recital = df.query("`place`=='리사이틀홀'")

In [5]:
df_concert.seat.sample(10)
# 3층 N블록1열 9 | 합창석 F블록4열 7 | 2층 BOX1 4 와 같이 split으로 3묶음으로 나뉘며, '블록'을 기준으로 나누어 담으면 될듯함. BOX는 '블록'으로 취급하기로 함.

357986      1층 D블록10열 8
24510        3층 A블록5열 5
1477862      2층 E블록4열 2
140283      2층 A블록7열 16
1715212      2층 E블록5열 5
1336245    1층 D블록21열 13
35284        2층 C블록6열 7
1790989     1층 E블록18열 4
822405       1층 E블록6열 5
514207      1층 B블록22열 8
Name: seat, dtype: object

In [12]:
# 'seat_floor', 'seat_block', 'seat_col', 'seat_num' 생성
df_concert1 = df_concert.copy() #실험용

# 1.seat_floor 생성
df_concert1['seat_floor'] = df_concert1['seat'].str.split(" ").str[0]
df_concert1['seat_floor'] = df_concert1['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제
df_concert1['seat_floor'] = df_concert1['seat_floor'].replace("합창석",4,regex=True) # 3층까지만 있으므로, 합창석은 4로 대체
df_concert1['seat_floor'].astype(int)

# 2. seat_block, seat_col 생성 
# 2-1. BOX1과 같이 "블록"으로 split 불가한 경우 대비 : seat_block에 넣어두고, 이 경우 seat_col은 nan이 되므로 -1 부여
df_concert1['seat_block'] = df_concert1['seat'].str.split(" ").str[1]
df_concert1['seat_col'] = -1

# 2-2. box df, block df 두 가지로 구분
df_concert_box = df_concert1.query('not seat_block.str.contains("블록")')
df_concert_block = df_concert1.query('seat_block.str.contains("블록")')

# 2-3. block df에서 "블록"기준으로 구분하여 담고, "열"글자 삭제
df_concert_block[['seat_block', 'seat_col']] = df_concert_block['seat_block'].str.split("블록", expand=True)
df_concert_block['seat_col'] = df_concert_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

df_concert2 = pd.concat([df_concert_box, df_concert_block])
df_concert2['seat_col'].astype(int)    
df_concert2

# 3. seat_num 생성
df_concert2['seat_num'] = df_concert2['seat'].str.split(" ").str[-1]
df_concert2['seat_num'].astype(int)

df_concert2

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,seat_floor,seat_block,seat_col,seat_num
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,1512,20220204,2000,3층 BOX9 10,10000,2,일반,302,20220114.0,20220115.0,교향곡,콘서트홀,120,15,Y,3,BOX9,-1,10
11,50.0,F,무료,NaN,NaN,NaN,NaN,NaN,20191005,1417,20191201,1700,3층 BOX9 5,24000,0,K-lang 멤버십 회원(1인2매)20%,315,NaN,20191005.0,오페라,콘서트홀,120,15,Y,3,BOX9,-1,5
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220624,1618,20220629,1930,3층 BOX12 3,20000,2,일반,2344,20220430.0,20220430.0,교향곡,콘서트홀,110,15,N,3,BOX12,-1,3
18,70.0,M,골드,무료,NaN,NaN,NaN,NaN,20191220,1106,20200211,1930,2층 BOX2 2,42000,0,골드회원 할인30%,76,20191220.0,20191223.0,클래식,콘서트홀,120,15,Y,2,BOX2,-1,2
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20230507,2130,20230701,1700,3층 BOX11 5,40000,0,일반,2822,20230505.0,20230507.0,교향곡,콘서트홀,120,15,N,3,BOX11,-1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920853,60.0,F,무료,그린,NaN,NaN,NaN,NaN,20210618,1509,20210704,1500,1층 A블록2열 2,90000,2,일반,887,20210618.0,20210620.0,교향곡,콘서트홀,110,20,Y,1,A,2,2
1920855,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20230522,1729,20230613,1700,3층 A블록6열 4,10000,0,일반,2825,20230505.0,20230507.0,교향곡,콘서트홀,120,15,Y,3,A,6,4
1920856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201009,1652,20201020,1930,1층 D블록16열 12,0,0,초대권,77,NaN,20200920.0,클래식,콘서트홀,120,15,N,1,D,16,12
1920858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200726,1655,20200818,1930,1층 D블록20열 8,0,0,기획사판매,850,20200719.0,20200720.0,합창,콘서트홀,110,15,N,1,D,20,8


In [14]:
df_concert2.isnull().sum() #null 값 없음 확인 완료!

age                   612451
gender                612101
membership_type_1     612101
membership_type_2     707231
membership_type_3     956500
membership_type_4    1071808
membership_type_5    1095858
membership_type_6    1096968
tran_date                  0
tran_time                  0
play_date                  0
play_st_time               0
seat                       0
price                      0
ticket_cancel              0
discount_type              0
performance_code           0
pre_open_date         395645
open_date                  0
genre                  25073
place                      0
running_time               0
intermission               0
member_yn                  0
seat_floor                 0
seat_block                 0
seat_col                   0
seat_num                   0
dtype: int64

In [73]:
df_chamber.seat.sample(10)
# 1층 B블록 1열 1 | 2층 BOX2 3 과 같음. 공백 기준 split으로 나눠담기.

1132844     1층 C블록 3열 2
1903235    1층 C블록 12열 1
591584     1층 B블록 13열 2
1706612    1층 B블록 14열 6
1756694    1층 B블록 9열 15
540888      1층 B블록 3열 7
474757      1층 A블록 8열 5
1910390     1층 B블록 7열 1
1252829    2층 B블록 4열 12
524961     1층 B블록 17열 7
Name: seat, dtype: object

In [78]:
df_recital.seat.sample(10)
#  1층 6열 2 |  1층 BOX1 2열 23 | 1층 우측 1열 7 | 2층 좌측 1열 9 와 같음.

1819125          1층 9열 1
712144          1층 12열 9
941151          1층 5열 13
385004          1층 7열 15
1807432    2층 BOX3 1열 17
1444448          1층 3열 4
718369          1층 2열 14
593094           1층 9열 5
523954         1층 10열 10
1037999       2층 좌측 1열 9
Name: seat, dtype: object

In [ ]:
# 블록은 BOXn, A,B,C 이런 식으로 되어 있음. 임의의 순서를 부여해서 나중에 알아보면 어떨까?